# GloVe Embeddings Bag - Experimento

A component that convert a document collection on GloVe Bag of Embeddings features
Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declare parâmetros e hiperparâmetros para o modelo
Os componentes podem declarar (e usar) estes parâmetros como padrão:
- dataset
- target

Use estes parâmetros para carregar/salvar conjutos de dados, modelos, métricas e figuras com a ajuda do [SDK da PlatIAgro](https://platiagro.github.io/sdk/). <br>
É possível também declarar parâmetros personalizados para serem definidos ao executar um experimento. 

Selecione os hiperparâmetros e seus respectivos valores para serem usados ao treinar o modelo:
- language

Estes parâmetros são alguns dos oferecidos pela classe do modelo, você também pode utilizar outros existentes. <br>
Dê uma olhada nos [parâmetros do modelo](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn-impute-simpleimputer) para mais informações.

In [19]:
# parâmetros
dataset = "imdb.csv" #@param {type:"string"}
target = "label" #@param {type:"feature", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}
text = "text" #@param {type:"string", label:"Texto alvo", description:"Nome da coluna do texto alvo pertencente ao dataset."}
language = "english" #@param ["portuguese", "english"] {type:"string", label:"Linguagem", description:"Linguagem da qual os stopwords pertencem. Deve ser a mesma utilizada no dataset."}
batch_size = 100

# selected features to perform the model
filter_type = "incluir" #@param ["incluir","remover"] {type:"string",multiple:false,label:"Modo de seleção das features",description:"Se deseja informar quais features deseja incluir no modelo, selecione a opção 'incluir'. Caso deseje informar as features que não devem ser utilizadas, selecione 'remover'. "}
model_features = "text" #@param {type:"feature",multiple:true,label:"Features para incluir/remover no modelo",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}

# features to apply One Hot Encoder
one_hot_features = "" #@param {type:"feature",multiple:true,label:"Features para fazer codificação one-hot", description: "Seu modelo utilizará a codificação one-hot para as features selecionadas. As demais features categóricas serão codificadas utilizando a codificação ordinal."}


## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [2]:
import pandas as pd

df = pd.read_csv(f'/tmp/data/{dataset}')
df

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False
...,...,...,...
995,negative,There are many different versions of this one ...,True
996,positive,Once upon a time Hollywood produced live-actio...,True
997,negative,Wenders was great with Million $ Hotel.I don't...,True
998,negative,Although a film with Bruce Willis is always wo...,True


## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [3]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Remoção de linhas com valores faltantes no atributo alvo

Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.

In [4]:
df.dropna(subset = [target],inplace=True)
y = df[target].to_numpy()

## Filtragem das features 

Seleciona apenas as features que foram declaradas no parâmetro model_features. Se nenhuma feature for especificada, todo o conjunto de dados será utilizado para a modelagem.

In [5]:
if filter_type == 'incluir':
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        featuretypes = featuretypes[columns_index]
    else:
        columns_to_filter = columns
else:
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns,columns_index)
        featuretypes = np.delete(featuretypes,columns_index)
    else:
        columns_to_filter = columns

# keep the features selected
df_model = df[columns_to_filter]
X = df_model.to_numpy()

## Divisão do datset em subconjuntos de treino e teste

Subconjunto de Treino: amostras de dados usado para treinar o modelo (``fit``). <br>
Subconjunto de Teste: a amostra de dados usada para fornecer uma avaliação imparcial de um modelo adequado ao conjunto de dados de treinamento.

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)

In [7]:
mapeamento = {'positive': True, 'negative': False}
y_train = [mapeamento.get(i, i) for i in y_train]
y_test = [mapeamento.get(i, i) for i in y_test]

## Busca por stopwords

Stopwords (ou palavras de parada) são palavras que geralmente se referem às mais comuns em um idioma ou em um corpus. <br>
Elas podem ser ignoradas com segurança sem sacrificar o significado da frase, pois são palarvas que não agregram muito significado.

In [8]:
!pip install nltk --quiet
import nltk

# Download stopwords from nltk
nltk.download('stopwords')

# Get a list of stopwords for the defined language
stopwords = nltk.corpus.stopwords.words(language)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Processamento do texto

Funções auxiliares para processamento dos dados.

In [9]:
from re import sub


def tokenize_without_punctuation(text_list: list = None):
    """Tokenize without ponctuation.

    Args:
        text_list (list): a list of texts to be used.

    Returns:
        A list of tokenized text without punctuation.
    """
    tokenize_list = list()
    punctuation_pattern = "[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]"
    html_tag_pattern = "<.*?>"

    for text in text_list:
        text = sub(html_tag_pattern, ' ', text[0])
        tokenize_list.append(sub(punctuation_pattern, ' ', text).split(' '))

    return tokenize_list

## Download dos Embeddings e Visualização do GloVe

Download dos Embeddings

In [10]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

!wget -nc http://nlp.stanford.edu/data/glove.6B.zip
!unzip -o glove.6B.zip -d glove_dir

DEBUG:root:test


File ‘glove.6B.zip’ already there; not retrieving.

Archive:  glove.6B.zip
  inflating: glove_dir/glove.6B.50d.txt  
  inflating: glove_dir/glove.6B.100d.txt  
  inflating: glove_dir/glove.6B.200d.txt  
  inflating: glove_dir/glove.6B.300d.txt  


Criação das Estruturas Glove

In [13]:
#Caso não possua instalado, instala antes de importar
try:
    from torchtext.vocab import GloVe
except ModuleNotFoundError:
    !pip install torchtext --quiet
    from torchtext.vocab import GloVe
    
#Definição das variáveis da Bag Of Embeddings
glove_dim = 300
glove = GloVe(name='6B', dim=glove_dim, cache='./glove_dir')
glove_vectors = glove.vectors
glove_vocab = glove.stoi
glove_words = glove.itos
print(glove.vectors.shape)
print('Primeiras 20 palavras e seus índices:', list(glove_vocab.items())[:20])
print('Primeiras 20 palavras:',glove_words[:20])
print('Primeiro vetor:', list(glove_vectors)[0])

INFO:torchtext.vocab:Loading vectors from ./glove_dir/glove.6B.300d.txt.pt


torch.Size([400000, 300])
Primeiras 20 palavras e seus índices: [('the', 0), (',', 1), ('.', 2), ('of', 3), ('to', 4), ('and', 5), ('in', 6), ('a', 7), ('"', 8), ("'s", 9), ('for', 10), ('-', 11), ('that', 12), ('on', 13), ('is', 14), ('was', 15), ('said', 16), ('with', 17), ('he', 18), ('as', 19)]
Primeiras 20 palavras: ['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s", 'for', '-', 'that', 'on', 'is', 'was', 'said', 'with', 'he', 'as']
Primeiro vetor: tensor([ 4.6560e-02,  2.1318e-01, -7.4364e-03, -4.5854e-01, -3.5639e-02,
         2.3643e-01, -2.8836e-01,  2.1521e-01, -1.3486e-01, -1.6413e+00,
        -2.6091e-01,  3.2434e-02,  5.6621e-02, -4.3296e-02, -2.1672e-02,
         2.2476e-01, -7.5129e-02, -6.7018e-02, -1.4247e-01,  3.8825e-02,
        -1.8951e-01,  2.9977e-01,  3.9305e-01,  1.7887e-01, -1.7343e-01,
        -2.1178e-01,  2.3617e-01, -6.3681e-02, -4.2318e-01, -1.1661e-01,
         9.3754e-02,  1.7296e-01, -3.3073e-01,  4.9112e-01, -6.8995e-01,
        -9.2462e-02,  2.

## Construção da matriz de Glove Embeddings Bag

In [14]:
import scipy

def build_glove_matrix(X,step,stopwords=stopwords):
    X = tokenize_without_punctuation(X)
    glove_matrix = []
    word_filtered_matrix = []
    for token_line in X:
        if step == 'train':
            token_phrase = [glove_vocab[word] for word in token_line if (word in glove_vocab and word not in stopwords)]
            filtered_words =  [word for word in token_line if (word in glove_vocab and word not in stopwords)]
        elif step == 'test':
            token_phrase = [glove_vocab[word] for word in token_line if (word in glove_vocab)]
            filtered_words = [word for word in token_line if (word in glove_vocab)]
        word_filtered_matrix.append(filtered_words)
        glove_matrix.append(token_phrase)
  
    return glove_matrix , word_filtered_matrix

#listas de listas construidas
X_train_glove_ids,X_train_glove_words  = build_glove_matrix(X_train,'train')
X_test_glove_ids,X_test_glove_words = build_glove_matrix(X_test,'test')

print(X_train_glove_ids[0])
print(X_train_glove_words[0])

[357, 328, 9679, 11089, 1005, 1592, 116, 46349, 662, 39086, 117, 530, 69, 1740, 2219, 2772, 222, 169, 3573, 11068, 720, 389, 389, 6614, 2532, 1005, 63173, 20333, 63903, 8589, 143, 2301, 81189, 1056, 1005, 116, 24056, 2905, 6616, 357, 328, 756, 254, 6107, 4881, 48, 988, 2543, 732, 779, 79, 8829, 26926, 48, 4662, 94, 3162, 137, 1502, 823, 7299, 47845, 357, 328, 756, 1005, 54, 1716, 1247, 24056, 2905, 357, 328, 1420]
['small', 'town', 'instantly', 'relate', 'movie', 'era', 'made', 'townsfolk', 'look', 'uncomfortably', 'like', 'lot', 'people', 'grew', 'plot', 'yes', 'going', 'get', 'nominated', 'anytime', 'soon', 'point', 'point', 'suspend', 'reality', 'movie', 'aplenty', 'greedy', 'uncaring', 'banker', 'well', 'meaning', 'dimwitted', 'deputy', 'movie', 'made', 'poke', 'fun', 'genre', 'small', 'town', 'living', 'best', 'smile', 'sight', 'one', 'growing', 'considering', 'technology', 'available', 'time', 'pleasant', 'romp', 'one', 'childhood', 'could', 'sit', 'back', 'afternoon', 'hand', 'l

## Criaçãod do Dataset

In [15]:
from torch.utils.data import Dataset
from numpy import genfromtxt
import torch

class ImdbDataset(Dataset):
    def __init__(self, X, X_words,target):
        super(ImdbDataset, self).__init__()

        self.x = [torch.tensor(line).type(torch.LongTensor) for line in X ]
        self.target = torch.tensor(target).type(torch.LongTensor) 
        self.words = X_words


    def __len__(self):
    
        return len(self.x)
  
    def __getitem__(self, index):
        return self.x[index], self.words[index], self.target[index]
    

Criando e Testando Datasets

In [16]:
torch_ds_train = ImdbDataset(X_train_glove_ids,X_train_glove_words,y_train)
torch_ds_test = ImdbDataset(X_test_glove_ids,X_test_glove_words,y_test)
print("------->Testando Dataset de treino<-------")
x1,xw1, y1 = torch_ds_train[0]
print(x1)
print(xw1)
print(y1)
print("------->Testando Dataset de teste<-------")
x2,xw2, y2 = torch_ds_test[0]
print(x2)
print(xw2)
print(y2)

------->Testando Dataset de treino<-------
tensor([  357,   328,  9679, 11089,  1005,  1592,   116, 46349,   662, 39086,
          117,   530,    69,  1740,  2219,  2772,   222,   169,  3573, 11068,
          720,   389,   389,  6614,  2532,  1005, 63173, 20333, 63903,  8589,
          143,  2301, 81189,  1056,  1005,   116, 24056,  2905,  6616,   357,
          328,   756,   254,  6107,  4881,    48,   988,  2543,   732,   779,
           79,  8829, 26926,    48,  4662,    94,  3162,   137,  1502,   823,
         7299, 47845,   357,   328,   756,  1005,    54,  1716,  1247, 24056,
         2905,   357,   328,  1420])
['small', 'town', 'instantly', 'relate', 'movie', 'era', 'made', 'townsfolk', 'look', 'uncomfortably', 'like', 'lot', 'people', 'grew', 'plot', 'yes', 'going', 'get', 'nominated', 'anytime', 'soon', 'point', 'point', 'suspend', 'reality', 'movie', 'aplenty', 'greedy', 'uncaring', 'banker', 'well', 'meaning', 'dimwitted', 'deputy', 'movie', 'made', 'poke', 'fun', 'genre', 

## Criando Dataloader

O Collatae serve par criar um offset em caso de entradas variáveis

In [17]:
def my_collate(batch):
    #import pdb;pdb.set_trace()
    # len soma de todas as palavras
    lista_words = []
    [lista_words.extend(item[1]) for item in batch]

    # len soma de todas as palavras
    lista_words_ids = [item[0] for item in batch]
    lista_words_ids_vector = torch.cat(lista_words_ids)

    #len batch_size
    target = [item[2] for item in batch]
    target = torch.stack(target)

    # len batch_size
    offsets = [0] + [len(entry) for entry in lista_words_ids]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    
    return lista_words_ids_vector, target , offsets


Criando Dataloaders

In [20]:
from torch.utils.data import DataLoader
train_loader = DataLoader(torch_ds_train, batch_size=batch_size, shuffle=True, collate_fn=my_collate)
test_loader = DataLoader(torch_ds_test, batch_size=batch_size, shuffle=False, collate_fn=my_collate)

Testando Dataloaders

In [21]:
print("------->Testando Dataloader de treino<-------")
next(iter(train_loader))

------->Testando Dataloader de treino<-------


(tensor([16058,   219,   333,  ...,  1588,   117,   456]),
 tensor([1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
         0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
         1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
         1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
         0, 0, 1, 1]),
 tensor([   0,  166,  278,  463,  687,  740,  789,  940, 1019, 1099, 1182, 1318,
         1375, 1588, 1658, 1685, 1790, 2156, 2246, 2297, 2680, 2794, 2844, 2923,
         3001, 3050, 3108, 3177, 3235, 3274, 3440, 3500, 3608, 3939, 4028, 4148,
         4205, 4247, 4315, 4378, 4570, 4597, 4612, 4687, 4845, 4879, 4930, 5304,
         5446, 5490, 5559, 5634, 5700, 5807, 5854, 5875, 5985, 6037, 6122, 6242,
         6335, 6444, 6525, 6580, 6635, 6691, 6828, 6932, 7026, 7086, 7116, 7296,
         7349, 7459, 7525, 7580, 7656, 7899, 7934, 7978, 8043, 8129, 8194, 8392,
         8584, 8644, 8757, 

In [22]:
print("------->Testando Dataloader de teste<-------")
next(iter(test_loader))

------->Testando Dataloader de teste<-------


(tensor([   14, 10221,    19,  ...,   308,    13,    20]),
 tensor([1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
         0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
         1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
         1, 0, 0, 0]),
 tensor([    0,   282,   458,   671,   870,  1128,  1238,  1400,  1675,  1805,
          1877,  2068,  2401,  2520,  2982,  3071,  3376,  3432,  3539,  3728,
          3877,  4086,  4450,  4725,  5174,  5220,  5265,  5887,  6376,  6780,
          7127,  7242,  7858,  7976,  8334,  8517,  8626,  9133,  9524,  9627,
          9956, 10107, 10460, 10601, 10745, 10845, 10986, 11192, 11335, 11436,
         11551, 11884, 12048, 12350, 12429, 12557, 12617, 12653, 12836, 12987,
         13117, 13403, 14176, 14229, 14334, 14461, 14510, 14673, 14792, 14826,
         15094, 15563, 15626, 15708, 1585

## Salva modelo e outros artefatos

Utiliza a função `save_model` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar modelos e outros artefatos.<br>
Essa função torna estes artefatos disponíveis para o notebook de implantação.

In [23]:
from platiagro import save_model

save_model(train_loader=train_loader,
           test_loader=test_loader)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): server.anonymous:80
DEBUG:urllib3.connectionpool:http://server.anonymous:80 "GET /notebook/anonymous/server/api/sessions HTTP/1.1" 200 1380
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): server.anonymous:80
DEBUG:urllib3.connectionpool:http://server.anonymous:80 "GET /notebook/anonymous/server/api/sessions HTTP/1.1" 200 1380
/opt/conda/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
DEBUG:urllib3.connectionpool:http://minio-service.kubeflow:9000 "PUT /anonymous/ HTTP/1.1" 409 None
DEBUG:urllib3.connectionpool:http://minio-service.kubeflow:9000 "PUT /anonymous/experiments/d930015c-45a2-42eb-912a-541ce198cee6/operators/fb743ef0-6ddd-4bc1-8d14-ec82ce0ef401/model.joblib HTTP/1.1" 200 0
